In [3]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# MNIST dataset
# train: (60000, 28, 28)
# train label: (60000,)
# test: (10000, 28, 28)
# test label: (10000,)
mnist = mnist

# load MNIST datset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1)) # (60000, 28, 28) -> (60000, 28, 28, 1)
test_images = test_images.reshape((10000, 28, 28, 1)) # (10000, 28, 28) -> (10000, 28, 28, 1)

train_labels = to_categorical(train_labels, 10) # (60000,) -> convert to one-hot vector -> (60000, 10)
test_labels = to_categorical(test_labels, 10) # (60000,) -> convert to one-hot vector -> (60000, 10)

# Normalize the pixels in 0.0~1.0 float
train_images, test_images = train_images / 255.0, test_images / 255.0

print("train images shape: ", train_images.shape)
print("train labels shape: ", train_labels.shape)

train images shape:  (60000, 28, 28, 1)
train labels shape:  (60000, 10)


In [4]:
# LeNet Model
# reference: "Gradient-based learning applied to document recognition[Lecun et al.]"
class mnist_lenet(tf.keras.Model):
    def __init__(self):
        super(mnist_lenet, self).__init__()
        self.conv1 = layers.Conv2D(filters=10, kernel_size=[3,3], input_shape = (28,28,1), activation= 'relu')
        self.pool1 = layers.MaxPooling2D(2, 2)
        self.conv2 = layers.Conv2D(filters=20, kernel_size=[3,3], activation= 'relu')
        self.pool2 = layers.MaxPooling2D(2, 2)
        self.conv3 = layers.Conv2D(filters=30, kernel_size=[3,3], activation= 'relu')
        self.flat = layers.Flatten()
        self.dense1 = layers.Dense(64, activation='relu')
        self.dense2 = layers.Dense(10, activation='softmax')


    def call(self, x):
        net = self.conv1(x)
        net = self.pool1(net)
        net = self.conv2(net)
        net = self.pool2(net)
        net = self.conv3(net)
        net = self.flat(net)
        net = self.dense1(net)
        net = self.dense2(net)
        return net

In [5]:
# Train & save model in frozen(.pb) format.
my_model = mnist_lenet()
my_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # learning configuration
my_model.fit(train_images, train_labels, batch_size=1, epochs=5, verbose=1) # learning (forward & bacwkard)
my_model.summary()
my_model.save('mnist_lenet') # model file will be saved in the directory where you opened this .ipynb file

Epoch 1/5


2024-06-29 13:57:26.948807: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


60000/60000 [==============================] - 61s 1ms/step - loss: 0.1298 - accuracy: 0.9601
Epoch 2/5
60000/60000 [==============================] - 62s 1ms/step - loss: 0.0727 - accuracy: 0.9801
Epoch 3/5
60000/60000 [==============================] - 61s 1ms/step - loss: 0.0677 - accuracy: 0.9813
Epoch 4/5
60000/60000 [==============================] - 60s 1ms/step - loss: 0.0685 - accuracy: 0.9822
Epoch 5/5
60000/60000 [==============================] - 61s 1ms/step - loss: 0.0647 - accuracy: 0.9836
Model: "mnist_lenet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  100       
                                                                 
 max_pooling2d (MaxPooling2D  multiple                 0         
 )                                                               
                                                                 
 conv2d

INFO:tensorflow:Assets written to: mnist_lenet/assets


INFO:tensorflow:Assets written to: mnist_lenet/assets


In [6]:
# Load model and convert it for TensorFlow Lite (.tflite format)
import tensorflow as tf
import numpy as np

model_path = './mnist_lenet/'
converter = tf.lite.TFLiteConverter.from_saved_model(model_path)
tflite_model = converter.convert()

2024-06-29 14:02:42.649243: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2024-06-29 14:02:42.649260: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2024-06-29 14:02:42.651578: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: ./mnist_lenet/
2024-06-29 14:02:42.653953: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-06-29 14:02:42.653959: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: ./mnist_lenet/
2024-06-29 14:02:42.657673: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2024-06-29 14:02:42.658543: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2024-06-29 14:02:42.710723: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: ./mnist_lenet/
2024-06-29 14

In [7]:
import pathlib

# Create folder to save model.
tflite_models_dir = pathlib.Path("./tflite_output/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the unquantized/float model:
tflite_model_file = tflite_models_dir/"mnist_lenet.tflite"
tflite_model_file.write_bytes(tflite_model)

105276

In [8]:
# For Edge TPU, must quntize the model in INT8 precision.

# Get representative data set for post-quantization.
# The representative data set prevents accuracy drop while quantization.
def representative_data_gen():
    for image in train_images[:1000]:  # Use a subset of the dataset
        # Resize the image to the input shape of LeNet model
        image = tf.image.resize(image, (28, 28))
        image = np.expand_dims(image, axis=0)
        yield [image]

model_path = './mnist_lenet/'
converter = tf.lite.TFLiteConverter.from_saved_model(model_path)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
tflite_model = converter.convert()

2024-06-29 14:02:51.717847: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2024-06-29 14:02:51.717867: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2024-06-29 14:02:51.717965: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: ./mnist_lenet/
2024-06-29 14:02:51.719297: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-06-29 14:02:51.719305: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: ./mnist_lenet/
2024-06-29 14:02:51.722660: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2024-06-29 14:02:51.768715: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: ./mnist_lenet/
2024-06-29 14:02:51.777397: I tensorflow/cc/saved_model/loader.cc:305] SavedModel load for tags { serve }; Status: success: OK. Took 59431 microse

In [10]:
import pathlib

# Create folder to save model.
tflite_models_dir = pathlib.Path("./tflite_output/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the unquantized/float model:
tflite_model_file = tflite_models_dir/"mnist_lenet_quant.tflite"
tflite_model_file.write_bytes(tflite_model)

32064